In [87]:
from google.colab import drive
drive.mount('/content/drive')

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [88]:
import pandas as pd

data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data_technicien.csv')
# data.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [89]:
# Séparer les données du technicien 1
data1 = data.drop(columns=['t2_casque', 't2_visiere', 't2_veste', 't2_col']).copy()

# Renommer les colonnes des données du technicien 1
data1.rename(
    columns={
        't1_casque': 'casque',
        't1_visiere': 'visiere',
        't1_veste': 'veste',
        't1_col': 'col',
        't1_gant_gauche': 'gant_gauche',
        't1_gant_droit': 'gant_droit'
    },
    inplace=True
)

# Séparer les données du technicien 2
data2 = data[['Scenario', 'Correct', 'Timestamp', 'Datetime', 't2_casque', 't2_visiere', 't2_veste', 't2_col', 'danger']].copy()

# Renommer les colonnes des données du technicien 2
data2.rename(
    columns={
        't2_casque': 'casque',
        't2_visiere': 'visiere',
        't2_veste': 'veste',
        't2_col': 'col'
    },
    inplace=True
)

# Afficher les colonnes des deux datasets
print(data1.columns)
print(data2.columns)

Index(['Scenario', 'Correct', 'Timestamp', 'Datetime', 'casque', 'visiere',
       'col', 'gant_gauche', 'gant_droit', 'danger'],
      dtype='object')
Index(['Scenario', 'Correct', 'Timestamp', 'Datetime', 'casque', 'visiere',
       'veste', 'col', 'danger'],
      dtype='object')


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [90]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

# Suppression des colonnes 'Datetime' et 'Timestamp'
data1_cleaned = data1.drop(columns=['Datetime', 'Timestamp'], errors='ignore')
data1_cleaned['Scenario'] = data1_cleaned['Scenario'].apply(lambda x: f"Scenario_{x}")

# Suppression des lignes contenant des NaN
data1_cleaned = data1_cleaned.dropna()

# Conversion en format transactionnel
transactions = data1_cleaned.apply(lambda row: row[row == 1].index.tolist() + [row['Scenario']], axis=1).tolist()

# Initialiser TransactionEncoder et transformer les données
te1 = TransactionEncoder()
te_data1 = te1.fit_transform(transactions)
te_data1 = pd.DataFrame(te_data1, columns=te1.columns_)

print('Colonne pour le technicien1 sont : ', te1.columns_)

# Extraction des itemsets fréquents avec FPGrowth ou Apriori
frequent_itemsets = fpgrowth(te_data1, min_support=0.05, use_colnames=True)
# print(frequent_itemsets[frequent_itemsets['itemsets'].apply(lambda x: 'danger' in x)])

# Générer les règles d'association
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=0.001, num_itemsets=len(frequent_itemsets))

# Filtrer les règles pour ne garder que celles où 'danger' est dans les conséquences
rules_danger = rules[rules['consequents'].apply(lambda x: 'danger' in x)]

# Afficher les règles filtrées
print(rules_danger[['antecedents', 'consequents', 'confidence', 'lift']].sort_values(by='confidence', ascending=False)[:15])


Colonne pour le technicien1 sont :  ['Correct', 'Scenario_1', 'Scenario_2', 'Scenario_3', 'Scenario_4', 'Scenario_6', 'casque', 'col', 'danger', 'gant_droit', 'gant_gauche', 'visiere']


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


                           antecedents                    consequents  \
300              (casque, gant_gauche)                       (danger)   
304                      (gant_gauche)               (casque, danger)   
296                      (gant_gauche)                       (danger)   
312          (gant_droit, gant_gauche)                       (danger)   
325          (gant_droit, gant_gauche)               (casque, danger)   
318  (casque, gant_droit, gant_gauche)                       (danger)   
322               (casque, gant_droit)          (gant_gauche, danger)   
298                       (gant_droit)                       (danger)   
329                       (gant_droit)  (casque, gant_gauche, danger)   
306               (casque, gant_droit)                       (danger)   
310                       (gant_droit)               (casque, danger)   
315                       (gant_droit)          (gant_gauche, danger)   
323              (casque, gant_gauche)           (g

In [91]:
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules, fpgrowth

# Suppression des colonnes 'Datetime' et 'Timestamp'
data2_cleaned = data2.drop(columns=['Datetime', 'Timestamp'], errors='ignore')
data2_cleaned['Scenario'] = data2_cleaned['Scenario'].apply(lambda x: f"Scenario_{x}")

# Suppression des lignes contenant des NaN
data2_cleaned = data2_cleaned.dropna()

# Conversion en format transactionnel
transactions2 = data2_cleaned.apply(lambda row: row[row == 1].index.tolist() + [row['Scenario']], axis=1).tolist()

# Initialiser TransactionEncoder et transformer les données
te2 = TransactionEncoder()
te_data2 = te2.fit_transform(transactions2)
te_data2 = pd.DataFrame(te_data2, columns=te2.columns_)

print('Colonne pour le technicien2 sont : ', te2.columns_)

# Extraction des itemsets fréquents avec FPGrowth ou Apriori
frequent_itemsets2 = fpgrowth(te_data2, min_support=0.05, use_colnames=True)
# print(frequent_itemsets2[frequent_itemsets2['itemsets'].apply(lambda x: 'danger' in x)])

# Générer les règles d'association
rules2 = association_rules(frequent_itemsets2, metric="lift", min_threshold=0.001, num_itemsets=len(frequent_itemsets2))

# Filtrer les règles pour ne garder que celles où 'danger' est dans les conséquences
rules_danger2 = rules2[rules2['consequents'].apply(lambda x: 'danger' in x)]

# Afficher les règles filtrées
print(rules_danger2[['antecedents', 'consequents', 'confidence', 'lift']].sort_values(by='confidence', ascending=False)[:15])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Colonne pour le technicien2 sont :  ['Correct', 'Scenario_1', 'Scenario_2', 'Scenario_5', 'casque', 'col', 'danger', 'veste', 'visiere']
                   antecedents            consequents  confidence      lift
415  (casque, col, Scenario_2)               (danger)    0.416667  1.944444
409       (casque, Scenario_2)               (danger)    0.387097  1.806452
429          (col, Scenario_2)               (danger)    0.361111  1.685185
401               (Scenario_2)               (danger)    0.333333  1.555556
420       (casque, Scenario_2)          (col, danger)    0.322581  3.387097
433               (Scenario_2)          (col, danger)    0.288889  3.033333
422          (col, Scenario_2)       (casque, danger)    0.277778  3.111111
413               (Scenario_2)       (casque, danger)    0.266667  2.986667
427               (Scenario_2)  (casque, col, danger)    0.222222  3.111111
394               (Scenario_1)               (danger)    0.215190  1.004219
402              (casque, c

In [92]:
data12 = pd.concat([data1.dropna(), data2.dropna()], ignore_index=True)

# Mettre veste a 1 si col est a 1
data12['veste'] = data12.apply(lambda row: 1 if row['col'] == 1 else row['veste'], axis=1)
data12.sort_values(by='Datetime', inplace=True)
data12.reset_index(drop=True, inplace=True)

data12



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Scenario,Correct,Timestamp,Datetime,casque,visiere,col,gant_gauche,gant_droit,danger,veste
0,1,1,1626084505,2021-07-12 10:08:25,1.0,0.0,1.0,NaN,NaN,0,1.0
1,1,1,1626084530,2021-07-12 10:08:50,1.0,0.0,0.0,NaN,NaN,0,0.0
2,1,1,1626084556,2021-07-12 10:09:16,1.0,0.0,0.0,NaN,NaN,0,0.0
3,1,1,1626084853,2021-07-12 10:14:13,1.0,0.0,0.0,NaN,NaN,0,1.0
4,1,1,1626084878,2021-07-12 10:14:38,1.0,0.0,1.0,NaN,NaN,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
388,5,0,1626965720,2021-07-22 14:55:20,0.0,0.0,0.0,NaN,NaN,0,0.0
389,5,0,1626965736,2021-07-22 14:55:36,1.0,0.0,0.0,NaN,NaN,0,0.0
390,5,0,1626965751,2021-07-22 14:55:51,1.0,0.0,0.0,NaN,NaN,0,0.0
391,5,0,1626965766,2021-07-22 14:56:06,1.0,0.0,0.0,NaN,NaN,0,0.0


In [93]:
# Suppression des colonnes 'Datetime' et 'Timestamp'
data12_cleaned = data12.drop(columns=['Datetime', 'Timestamp'], errors='ignore')
data12_cleaned['Scenario'] = data12_cleaned['Scenario'].apply(lambda x: f"Scenario_{x}")


# Conversion en format transactionnel
transactions12 = data12_cleaned.apply(lambda row: row[row == 1].index.tolist() + [row['Scenario']], axis=1).tolist()

# Initialiser TransactionEncoder et transformer les données
te12 = TransactionEncoder()
te_data12 = te12.fit_transform(transactions12)
te_data12 = pd.DataFrame(te_data12, columns=te12.columns_)

print('Colonne pour les technicien 1 et 2 sont : ', te12.columns_)

# Extraction des itemsets fréquents avec FPGrowth ou Apriori
frequent_itemsets12 = fpgrowth(te_data12, min_support=0.05, use_colnames=True)
# print(frequent_itemsets12)
# print(frequent_itemsets12[frequent_itemsets12['itemsets'].apply(lambda x: 'danger' in x)])

# Générer les règles d'association
rules12 = association_rules(frequent_itemsets12, metric="lift", min_threshold=0.01, num_itemsets=len(frequent_itemsets12))

# Filtrer les règles pour ne garder que celles où 'danger' est dans les conséquences
rules_danger12 = rules12[rules12['consequents'].apply(lambda x: 'danger' in x)]

# Afficher les règles filtrées
print(rules_danger12[['antecedents', 'consequents', 'confidence', 'lift']].sort_values(by='confidence', ascending=False)[:15])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Colonne pour les technicien 1 et 2 sont :  ['Correct', 'Scenario_1', 'Scenario_2', 'Scenario_3', 'Scenario_4', 'Scenario_5', 'Scenario_6', 'casque', 'col', 'danger', 'gant_droit', 'gant_gauche', 'veste', 'visiere']
                           antecedents                    consequents  \
446                      (gant_gauche)                       (danger)   
462              (casque, gant_gauche)                       (danger)   
466                      (gant_gauche)               (casque, danger)   
556  (casque, gant_droit, gant_gauche)                       (danger)   
563          (gant_droit, gant_gauche)               (casque, danger)   
544          (gant_droit, gant_gauche)                       (danger)   
567                       (gant_droit)  (casque, gant_gauche, danger)   
482                       (gant_droit)                       (danger)   
547                       (gant_droit)          (gant_gauche, danger)   
550               (casque, gant_droit)                 

In [94]:
# Exporter en csv
rules_danger.to_csv('rules_danger.csv', index=False)
rules_danger2.to_csv('rules_danger2.csv', index=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [95]:
data12

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Scenario,Correct,Timestamp,Datetime,casque,visiere,col,gant_gauche,gant_droit,danger,veste
0,1,1,1626084505,2021-07-12 10:08:25,1.0,0.0,1.0,NaN,NaN,0,1.0
1,1,1,1626084530,2021-07-12 10:08:50,1.0,0.0,0.0,NaN,NaN,0,0.0
2,1,1,1626084556,2021-07-12 10:09:16,1.0,0.0,0.0,NaN,NaN,0,0.0
3,1,1,1626084853,2021-07-12 10:14:13,1.0,0.0,0.0,NaN,NaN,0,1.0
4,1,1,1626084878,2021-07-12 10:14:38,1.0,0.0,1.0,NaN,NaN,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
388,5,0,1626965720,2021-07-22 14:55:20,0.0,0.0,0.0,NaN,NaN,0,0.0
389,5,0,1626965736,2021-07-22 14:55:36,1.0,0.0,0.0,NaN,NaN,0,0.0
390,5,0,1626965751,2021-07-22 14:55:51,1.0,0.0,0.0,NaN,NaN,0,0.0
391,5,0,1626965766,2021-07-22 14:56:06,1.0,0.0,0.0,NaN,NaN,0,0.0


**Augmentation des données**

In [96]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

# Séparer les données en X (features) et y (labels)
data12 = data12.fillna(0)
X, y = data12.drop(['danger', 'Datetime'], axis=1), data12['danger']

non_numeric_columns = X.select_dtypes(include=['object']).columns

from sklearn.preprocessing import LabelEncoder

# Appliquer un encodage sur les colonnes catégoriques
for col in non_numeric_columns:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])


# Appliquer SMOTE sur les données préparées
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Vérifier la répartition après SMOTE
from collections import Counter
print(f"Distribution après SMOTE : {Counter(y_resampled)}")

X_resampled = X_resampled.round()

print(pd.concat([X_resampled, y_resampled], axis=1))


Distribution après SMOTE : Counter({0: 333, 1: 333})
     Scenario  Correct   Timestamp  casque  visiere  col  gant_gauche  \
0           1        1  1626084505     1.0      0.0  1.0          0.0   
1           1        1  1626084530     1.0      0.0  0.0          0.0   
2           1        1  1626084556     1.0      0.0  0.0          0.0   
3           1        1  1626084853     1.0      0.0  0.0          0.0   
4           1        1  1626084878     1.0      0.0  1.0          0.0   
..        ...      ...         ...     ...      ...  ...          ...   
661         1        0  1626087267     0.0      0.0  0.0          0.0   
662         3        0  1626099856     1.0      0.0  0.0          1.0   
663         1        0  1626087386     0.0      0.0  0.0          0.0   
664         1        0  1626087274     0.0      0.0  0.0          0.0   
665         3        0  1626099815     1.0      1.0  0.0          1.0   

     gant_droit  veste  danger  
0           0.0    1.0       0  
1   

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/utils/_tags.py:354: FutureWarning: The SMOTE or classes from which it inherits use `_get_tags` and `_more_tags`. Please define the `__sklearn_tags__` method, or inherit from `sklearn.base.BaseEstimator` and/or other appropriate mixins such as `sklearn.base.TransformerMixin`, `sklearn.base.Clas

In [97]:
# Suppression des colonnes 'Datetime' et 'Timestamp'
data12_cleaned = pd.concat([X_resampled, y_resampled], axis=1).drop(columns=['Datetime'], errors='ignore')
data12_cleaned['Scenario'] = data12_cleaned['Scenario'].apply(lambda x: f"Scenario_{x}")


# Conversion en format transactionnel
transactions12 = data12_cleaned.apply(lambda row: row[row == 1].index.tolist() + [row['Scenario']], axis=1).tolist()

# Initialiser TransactionEncoder et transformer les données
te12 = TransactionEncoder()
te_data12 = te12.fit_transform(transactions12)
te_data12 = pd.DataFrame(te_data12, columns=te12.columns_)

print('Colonne pour les technicien 1 et 2 sont : ', te12.columns_)

# Extraction des itemsets fréquents avec FPGrowth ou Apriori
frequent_itemsets12 = fpgrowth(te_data12, min_support=0.1, use_colnames=True)
# print(frequent_itemsets12)
# print(frequent_itemsets12[frequent_itemsets12['itemsets'].apply(lambda x: 'danger' in x)])

# Générer les règles d'association
rules12 = association_rules(frequent_itemsets12, metric="lift", min_threshold=0.1, num_itemsets=len(frequent_itemsets12))

# Filtrer les règles pour ne garder que celles où 'danger' est dans les conséquences
rules_danger12 = rules12[rules12['consequents'].apply(lambda x: 'danger' in x)]

# Afficher les règles filtrées
# print(rules_danger12[['antecedents', 'consequents', 'confidence', 'lift']].sort_values(by='confidence', ascending=False)[:15])

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Colonne pour les technicien 1 et 2 sont :  ['Correct', 'Scenario_1', 'Scenario_2', 'Scenario_3', 'Scenario_4', 'Scenario_5', 'Scenario_6', 'casque', 'col', 'danger', 'gant_droit', 'gant_gauche', 'veste', 'visiere']


In [98]:
# Supposons un dataframe `rules` avec les colonnes 'support', 'confidence', et 'lift'
best_rules = rules_danger12[(rules_danger12['support'] >= 0.05) &
                   (rules_danger12['confidence'] >= 0.8) &
                   (rules_danger12['lift'] > 1.5)]

best_rules.sort_values(by=['lift', 'confidence'], ascending=False)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
195,"(casque, veste, Scenario_2)","(col, danger)",0.126126,0.136637,0.103604,0.821429,6.011774,1.0,0.086370,4.834835,0.953982,0.650943,0.793168,0.789835
198,"(casque, col, Scenario_2)","(veste, danger)",0.126126,0.136637,0.103604,0.821429,6.011774,1.0,0.086370,4.834835,0.953982,0.650943,0.793168,0.789835
54,"(veste, casque, Scenario_2)",(danger),0.126126,0.500000,0.103604,0.821429,1.642857,1.0,0.040541,2.800000,0.447781,0.198276,0.642857,0.514318
176,"(casque, col, Scenario_2)",(danger),0.126126,0.500000,0.103604,0.821429,1.642857,1.0,0.040541,2.800000,0.447781,0.198276,0.642857,0.514318
190,"(casque, veste, col, Scenario_2)",(danger),0.126126,0.500000,0.103604,0.821429,1.642857,1.0,0.040541,2.800000,0.447781,0.198276,0.642857,0.514318


In [99]:
# Calcul d'un score composite
best_rules['score'] = (0.5 * best_rules['confidence'] + 0.5 * best_rules['lift'])
best_rules.sort_values(by='score', ascending=False)


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-99-cded291a0502>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  best_rules['score'] = (0.5 * best_rules['confidence'] + 0.5 * best_rules['lift'])


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,score
195,"(casque, veste, Scenario_2)","(col, danger)",0.126126,0.136637,0.103604,0.821429,6.011774,1.0,0.086370,4.834835,0.953982,0.650943,0.793168,0.789835,3.416601
198,"(casque, col, Scenario_2)","(veste, danger)",0.126126,0.136637,0.103604,0.821429,6.011774,1.0,0.086370,4.834835,0.953982,0.650943,0.793168,0.789835,3.416601
54,"(veste, casque, Scenario_2)",(danger),0.126126,0.500000,0.103604,0.821429,1.642857,1.0,0.040541,2.800000,0.447781,0.198276,0.642857,0.514318,1.232143
176,"(casque, col, Scenario_2)",(danger),0.126126,0.500000,0.103604,0.821429,1.642857,1.0,0.040541,2.800000,0.447781,0.198276,0.642857,0.514318,1.232143
190,"(casque, veste, col, Scenario_2)",(danger),0.126126,0.500000,0.103604,0.821429,1.642857,1.0,0.040541,2.800000,0.447781,0.198276,0.642857,0.514318,1.232143


In [100]:
print(pd.concat([X_resampled, y_resampled], axis=1))

     Scenario  Correct   Timestamp  casque  visiere  col  gant_gauche  \
0           1        1  1626084505     1.0      0.0  1.0          0.0   
1           1        1  1626084530     1.0      0.0  0.0          0.0   
2           1        1  1626084556     1.0      0.0  0.0          0.0   
3           1        1  1626084853     1.0      0.0  0.0          0.0   
4           1        1  1626084878     1.0      0.0  1.0          0.0   
..        ...      ...         ...     ...      ...  ...          ...   
661         1        0  1626087267     0.0      0.0  0.0          0.0   
662         3        0  1626099856     1.0      0.0  0.0          1.0   
663         1        0  1626087386     0.0      0.0  0.0          0.0   
664         1        0  1626087274     0.0      0.0  0.0          0.0   
665         3        0  1626099815     1.0      1.0  0.0          1.0   

     gant_droit  veste  danger  
0           0.0    1.0       0  
1           0.0    0.0       0  
2           0.0    0.0  

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [101]:
X.dtypes

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,0
Scenario,int64
Correct,int64
Timestamp,int64
casque,float64
visiere,float64
col,float64
gant_gauche,float64
gant_droit,float64
veste,float64


In [103]:
import pandas as pd

df = pd.concat([X, y], axis=1)

# Ensure 'Timestamp' is of datetime dtype
# df['Timestamp'] = pd.to_datetime(df['Timestamp'])

# Calculer la différence entre les timestamps pour chaque scénario
df['Duree'] = df.groupby('Scenario')['Timestamp'].diff().fillna(0).astype(int)

# Supprimer timestamp
df = df.drop(columns=['Timestamp'])

X, y = df.drop(['danger'], axis=1), df['danger']

print(pd.concat([X, y], axis=1))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


KeyError: 'Column not found: Timestamp'

In [104]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Correctly assign the outputs of train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Accuracy: 0.9620253164556962
              precision    recall  f1-score   support

           0       1.00      0.95      0.98        66
           1       0.81      1.00      0.90        13

    accuracy                           0.96        79
   macro avg       0.91      0.98      0.94        79
weighted avg       0.97      0.96      0.96        79

[[63  3]
 [ 0 13]]


In [109]:
from sklearn.cluster import KMeans
import numpy as np

# Convertir en tableau np
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model_kmeans = KMeans(n_clusters=2, random_state=42)
model_kmeans.fit(X_train)

# Remove the parentheses from X_test
y_pred_kmeans = model_kmeans.predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred_kmeans)}")
print(classification_report(y_test, y_pred_kmeans))

Accuracy: 0.8354430379746836
              precision    recall  f1-score   support

           0       0.84      1.00      0.91        66
           1       0.00      0.00      0.00        13

    accuracy                           0.84        79
   macro avg       0.42      0.50      0.46        79
weighted avg       0.70      0.84      0.76        79



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/

In [118]:
from sklearn.cluster import HDBSCAN

model_hdbscan = HDBSCAN(min_cluster_size=20)
model_hdbscan.fit(X_train)

y_pred_hdbscan = model_hdbscan.fit_predict(X_test)

print(f"Accuracy: {accuracy_score(y_test, y_pred_hdbscan)}")
print(classification_report(y_test, y_pred_hdbscan))

Accuracy: 0.4177215189873418
              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.73      0.48      0.58        66
           1       0.03      0.08      0.05        13

    accuracy                           0.42        79
   macro avg       0.25      0.19      0.21        79
weighted avg       0.61      0.42      0.49        79



/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/d